To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!
<div class="align-center">
<a href="https://unsloth.ai/"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
<a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
<a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a></a> Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

To install Unsloth on your own computer, follow the installation instructions on our Github page [here](https://docs.unsloth.ai/get-started/installing-+-updating).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save)


### News

**Read our [blog post](https://unsloth.ai/blog/r1-reasoning) for guidance on how to train reasoning models.**

Visit our docs for all our [model uploads](https://docs.unsloth.ai/get-started/all-our-models) and [notebooks](https://docs.unsloth.ai/get-started/unsloth-notebooks).


### Installation

In [1]:
%%capture
# Skip restarting message in Colab
import sys; modules = list(sys.modules.keys())
for x in modules: sys.modules.pop(x) if "PIL" in x or "google" in x else None

!pip install unsloth vllm
!pip install --upgrade pillow
# If you are running this notebook on local, you need to install `diffusers` too
# !pip install diffusers
# Temporarily install a specific TRL nightly version
!pip install git+https://github.com/huggingface/trl.git@e95f9fb74a3c3647b86f251b7e230ec51c64b72b

### Unsloth

Use `PatchFastRL` before all functions to patch GRPO and other RL algorithms!

In [2]:
from unsloth import FastLanguageModel, PatchFastRL
PatchFastRL("GRPO", FastLanguageModel)

Unsloth: Patching Xformers to fix some performance issues.
🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 02-11 17:45:10 __init__.py:190] Automatically detected platform cuda.


Load up `Qwen 2.5 3B Instruct`, and set parameters

In [7]:
from unsloth import is_bfloat16_supported
import torch
max_seq_length = 32000 # Can increase for longer reasoning traces
lora_rank = 64 # Larger rank = smarter, but slower

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "meta-llama/meta-Llama-3.1-8B-Instruct",
    max_seq_length = max_seq_length,
    load_in_4bit = True, # False for LoRA 16bit
    fast_inference = True, # Enable vLLM fast inference
    max_lora_rank = lora_rank,
    gpu_memory_utilization = 0.5, # Reduce if out of memory
)

model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ], # Remove QKVO if out of memory
    lora_alpha = lora_rank,
    use_gradient_checkpointing = "unsloth", # Enable long context finetuning
    random_state = 3407,
)

==((====))==  Unsloth 2025.2.5: Fast Qwen2 patching. Transformers: 4.48.2.
   \\   /|    GPU: NVIDIA A100-SXM4-40GB. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading unsloth/qwen2.5-3b-instruct-unsloth-bnb-4bit with actual GPU utilization = 15.92%
Unsloth: Your GPU has CUDA compute capability 8.0 with VRAM = 39.56 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill tokens = 32000. Num Sequences = 160.
Unsloth: vLLM's KV Cache can use up to 3.88 GB. Also swap space = 6 GB.
INFO 02-11 17:55:34 config.py:542] This model supports multiple tasks: {'generate', 'embed', 'reward', 'classify', 'score'}. Defaulting to 'generate'.
Unsloth: vLLM Bitsandbytes config using kwa

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


INFO 02-11 17:55:39 model_runner.py:1115] Loading model weights took 2.2082 GB
INFO 02-11 17:55:41 worker.py:267] Memory profiling takes 2.17 seconds
INFO 02-11 17:55:41 worker.py:267] the current vLLM instance can use total_gpu_memory (39.56GiB) x gpu_memory_utilization (0.16) = 6.30GiB
INFO 02-11 17:55:41 worker.py:267] model weights take 2.21GiB; non_torch_memory takes 0.00GiB; PyTorch activation peak memory takes 2.62GiB; the rest of the memory reserved for KV Cache is 1.47GiB.
INFO 02-11 17:55:42 executor_base.py:110] # CUDA blocks: 2674, # CPU blocks: 10922
INFO 02-11 17:55:42 executor_base.py:115] Maximum concurrency for 32000 tokens per request: 1.34x
INFO 02-11 17:55:47 model_runner.py:1434] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error occurs during cudagraph capture, consider decreasing `gpu_memory_util

Capturing CUDA graph shapes: 100%|██████████| 23/23 [00:41<00:00,  1.83s/it]

INFO 02-11 17:56:29 model_runner.py:1562] Graph capturing finished in 42 secs, took 0.54 GiB
INFO 02-11 17:56:29 llm_engine.py:431] init engine (profile, create kv cache, warmup model) took 50.23 seconds


### Data Prep
<a name="Data"></a>

We directly leverage [@willccbb](https://gist.github.com/willccbb/4676755236bb08cab5f4e54a0475d6fb) for data prep and all reward functions. You are free to create your own!

In [36]:
import re
from datasets import load_dataset, Dataset

# Load and prep dataset
SYSTEM_PROMPT = """
Expected answer format:
```git-commit-message
The git commit title (max 60-character)

The git commit description (Only readable text)
```

You must avoid prefix on title.
You are an expert developer, so you know how to read all kinds of code syntax.
Read the git patch diff calmly from top to bottom, paying attention to each addition, deletion, and unchanged line carefully.
Focus on changes, not only the last or first lines, and figure out the main idea of the input.
If complex, break it down into smaller parts to organize your thoughts.
If JSON or declaration structures are present, pay attention to the special case on JSONs diff to avoid misinterpretation, but if it's a regular code, focus on the context and the changes made. Write a commit message based on the git diff provided.
Read the diff and write a commit message that accurately describes the changes made.
Focus in write why instead of what on your description message.
Your message is to add a kind of explanation for future consulting, so be clear and concise.

MANDATORY: Respond in the following format:
<reasoning>
...
</reasoning>
<answer>
...
</answer>
"""

XML_COT_FORMAT = """\
<reasoning>
{reasoning}
</reasoning>
<answer>
{answer}
</answer>
"""

def extract_xml_answer(text: str) -> str:
    answer = text.split("<answer>")[-1]
    answer = answer.split("</answer>")[0]
    return answer.strip()

# uncomment middle messages for 1-shot prompting
def get_gsm8k_questions(split = "train") -> Dataset:
    data = load_dataset('Tavernari/git-commit-message', 'default')[split] # type: ignore
    data = data.map(lambda x: { # type: ignore
        'prompt': [
            {'role': 'system', 'content': SYSTEM_PROMPT},
            {'role': 'user', 'content': x['input']}
        ],
        'answer': x['output']
    }) # type: ignore
    return data # type: ignore

dataset = get_gsm8k_questions()

# Reward functions
def correctness_reward_func(prompts, completions, answer, **kwargs) -> list[float]:
    responses = [completion[0]['content'] for completion in completions]
    q = prompts[0][-1]['content']
    extracted_responses = [extract_xml_answer(r) for r in responses]
    print('-'*20, f"Question:\n{q}", f"\nAnswer:\n{answer[0]}", f"\nResponse:\n{responses[0]}", f"\nExtracted:\n{extracted_responses[0]}")
    return [2.0 if r == a else 0.0 for r, a in zip(extracted_responses, answer)]

def int_reward_func(completions, **kwargs) -> list[float]:
    responses = [completion[0]['content'] for completion in completions]
    extracted_responses = [extract_xml_answer(r) for r in responses]
    return [0.5 if r.isdigit() else 0.0 for r in extracted_responses]

def strict_format_reward_func(completions, **kwargs) -> list[float]:
    """Reward function that checks if the completion has a specific format."""
    pattern = r"^<reasoning>\n.*?\n</reasoning>\n<answer>\n.*?\n</answer>\n$"
    responses = [completion[0]["content"] for completion in completions]
    matches = [re.match(pattern, r) for r in responses]
    return [0.5 if match else 0.0 for match in matches]

def soft_format_reward_func(completions, **kwargs) -> list[float]:
    """Reward function that checks if the completion has a specific format."""
    pattern = r"<reasoning>.*?</reasoning>\s*<answer>.*?</answer>"
    responses = [completion[0]["content"] for completion in completions]
    matches = [re.match(pattern, r) for r in responses]
    return [0.5 if match else 0.0 for match in matches]

def count_xml(text) -> float:
    count = 0.0
    if text.count("<reasoning>\n") == 1:
        count += 0.125
    if text.count("\n</reasoning>\n") == 1:
        count += 0.125
    if text.count("\n<answer>\n") == 1:
        count += 0.125
        count -= len(text.split("\n</answer>\n")[-1])*0.001
    if text.count("\n</answer>") == 1:
        count += 0.125
        count -= (len(text.split("\n</answer>")[-1]) - 1)*0.001
    return count

def xmlcount_reward_func(completions, **kwargs) -> list[float]:
    contents = [completion[0]["content"] for completion in completions]
    return [count_xml(c) for c in contents]

Map:   0%|          | 0/2535 [00:00<?, ? examples/s]

<a name="Train"></a>
### Train the model

Now set up GRPO Trainer and all configurations!

In [42]:
from trl import GRPOConfig, GRPOTrainer
training_args = GRPOConfig(
    use_vllm = True, # use vLLM for fast inference!
    learning_rate = 5e-6,
    adam_beta1 = 0.9,
    adam_beta2 = 0.99,
    weight_decay = 0.1,
    warmup_ratio = 0.1,
    lr_scheduler_type = "cosine",
    optim = "adamw_8bit",
    logging_steps = 1,
    bf16 = is_bfloat16_supported(),
    fp16 = not is_bfloat16_supported(),
    per_device_train_batch_size = 1,
    gradient_accumulation_steps = 1, # Increase to 4 for smoother training
    num_generations = 8, # Decrease if out of memory
    max_prompt_length = 1024,
    max_completion_length = 1024,
    # num_train_epochs = 1, # Set to 1 for a full training run
    max_steps = 250,
    save_steps = 250,
    max_grad_norm = 0.1,
    report_to = "none", # Can use Weights & Biases
    output_dir = "outputs",
)

torch.distributed process group is initialized, but parallel_mode != ParallelMode.DISTRIBUTED. In order to use Torch DDP, launch your script with `python -m torch.distributed.launch


And let's run the trainer! If you scroll up, you'll see a table of rewards. The goal is to see the `reward` column increase!

You might have to wait 150 to 200 steps for any action. You'll probably get 0 reward for the first 100 steps. Please be patient!

| Step | Training Loss | reward    | reward_std | completion_length | kl       |
|------|---------------|-----------|------------|-------------------|----------|
| 1    | 0.000000      | 0.125000  | 0.000000   | 200.000000        | 0.000000 |
| 2    | 0.000000      | 0.072375  | 0.248112   | 200.000000        | 0.000000 |
| 3    | 0.000000      | -0.079000 | 0.163776   | 182.500000        | 0.000005 |


In [ ]:
trainer = GRPOTrainer(
    model = model,
    processing_class = tokenizer,
    reward_funcs = [
        xmlcount_reward_func,
        soft_format_reward_func,
        strict_format_reward_func,
        int_reward_func,
        correctness_reward_func,
    ],
    args = training_args,
    train_dataset = dataset,
)
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 2,535 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 1 | Gradient Accumulation steps = 1
\        /    Total batch size = 1 | Total steps = 500
 "-____-"     Number of trainable parameters = 119,734,272


-------------------- Question:
diff --git a/src/main/kotlin/math/TwoSum.kt b/src/main/kotlin/math/TwoSum.kt
new file mode 100644
index 0000000..7b18f4d
--- /dev/null
+++ b/src/main/kotlin/math/TwoSum.kt
@@ -0,0 +1,19 @@
+package math
+/**
+ * Try all the pairs in the array and see if any of them add up to the target number.
+ * @param nums Array of integers.
+ * @param target Integer target.
+ * @return Indices of the two numbers such that they add up to target.
+ */
+fun twoSum(nums: IntArray, target: Int): IntArray{
+    for (index1 in nums.indices) {
+        val startIndex = index1 + 1
+        for (index2 in startIndex..nums.lastIndex) {
+            if (nums[index1] + nums[index2] == target) {
+                return intArrayOf(index1, index2)
+            }
+        }
+    }
+    return intArrayOf(0,1)
+
+}
\ No newline at end of file 
Answer:
```git-commit-message
Add TwoSum function for finding indices of two numbers that sum to target

This commit introduces the `twoSum` func

Step,Training Loss,reward,reward_std,completion_length,kl
1,0.015600,0.316000,0.018315,46.875000,0.391115
2,0.014100,0.254625,0.083879,50.500000,0.351258
3,0.015700,0.335375,0.131017,39.875000,0.393112
4,0.201900,0.063250,0.225474,97.500000,5.047631
5,0.028100,0.346125,0.055383,35.625000,0.701894
6,0.014700,0.358125,0.035438,34.125000,0.368406
7,0.022200,0.358375,0.037370,34.125000,0.555060
8,0.120600,0.361000,0.027974,30.750000,3.015473
9,0.038600,0.191500,0.127824,60.000000,0.964635
10,0.027500,0.356125,0.035049,35.125000,0.688266


Streaming output truncated to the last 5000 lines.
 	}
 
 	caps := []Capability{CapabilityCompletion}
+	if req.Suffix != "" {
+		caps = append(caps, CapabilityInsert)
+	}
+
 	r, m, opts, err := s.scheduleRunner(c.Request.Context(), req.Model, caps, req.Options, req.KeepAlive)
 	if errors.Is(err, errCapabilityCompletion) {
 		c.JSON(http.StatusBadRequest, gin.H{"error": fmt.Sprintf("%q does not support generate", req.Model)})
@@ -150,19 +154,6 @@ func (s *Server) GenerateHandler(c *gin.Context) {
 
 	prompt := req.Prompt
 	if !req.Raw {
-		var msgs []api.Message
-		if req.System != "" {
-			msgs = append(msgs, api.Message{Role: "system", Content: req.System})
-		} else if m.System != "" {
-			msgs = append(msgs, api.Message{Role: "system", Content: m.System})
-		}
-
-		for _, i := range images {
-			msgs = append(msgs, api.Message{Role: "user", Content: fmt.Sprintf("[img-%d]", i.ID)})
-		}
-
-		msgs = append(msgs, api.Message{Role: "user", Content: req.Prompt})
-
 		tmpl := m.Template
 

Step,Training Loss,reward,reward_std,completion_length,kl
1,0.015600,0.316000,0.018315,46.875000,0.391115
2,0.014100,0.254625,0.083879,50.500000,0.351258
3,0.015700,0.335375,0.131017,39.875000,0.393112
4,0.201900,0.063250,0.225474,97.500000,5.047631
5,0.028100,0.346125,0.055383,35.625000,0.701894
6,0.014700,0.358125,0.035438,34.125000,0.368406
7,0.022200,0.358375,0.037370,34.125000,0.555060
8,0.120600,0.361000,0.027974,30.750000,3.015473
9,0.038600,0.191500,0.127824,60.000000,0.964635
10,0.027500,0.356125,0.035049,35.125000,0.688266


-------------------- Question:
diff --git a/JsonConfig.Tests/JSON/EnabledModules.json b/JsonConfig.Tests/JSON/EnabledModules.json
new file mode 100644
index 0000000..7d7ef7f
--- /dev/null
+++ b/JsonConfig.Tests/JSON/EnabledModules.json
@@ -0,0 +1,12 @@
+
+{
+	"EnabledModulesArray": [
+		"Module1",
+		"Module2"
+	],
+	"EnabledModulesObject" : {
+		"Module1" : true,
+		"Module2" : false
+	}
+	
+}
\ No newline at end of file
diff --git a/JsonConfig.Tests/JSON/Types.json b/JsonConfig.Tests/JSON/Types.json
new file mode 100644
index 0000000..22d0a96
--- /dev/null
+++ b/JsonConfig.Tests/JSON/Types.json
@@ -0,0 +1,27 @@
+{
+	"Foo" : "bar",
+	"NestedArray" : [
+		{ "Foo" : "bar" },	
+		{ "Foo" : "bar" },	
+		{ "Foo" : "bar" },	
+		{ "Foo" : "bar" }	
+	],
+	"DoubleNestedArray" : [
+		{
+			"One" : [
+				{ "Foo" : "bar" },	
+				{ "Foo" : "bar" },	
+				{ "Foo" : "bar" },	
+				{ "Foo" : "bar" }
+			]
+		},
+		{
+			"Two" : [
+				{ "Foo" : "bar" },	
+				{ "Foo" : "bar" },	
+				{ "Foo" : "bar

<a name="Inference"></a>
### Inference
Now let's try the model we just trained! First, let's first try the model without any GRPO trained:

In [40]:
text = tokenizer.apply_chat_template([
    {"role" : "user", "content" : "diff --git a/Project/Scenes/Browse/InteractiveSchedule/Vertical Collection/Browse.InteractiveSchedule.VerticalCollection+ViewController.swift b/Project/Scenes/Browse/InteractiveSchedule/Vertical Collection/Browse.InteractiveSchedule.VerticalCollection+ViewController.swift --- a/Project/Scenes/Browse/InteractiveSchedule/Vertical Collection/Browse.InteractiveSchedule.VerticalCollection+ViewController.swift +++ b/Project/Scenes/Browse/InteractiveSchedule/Vertical Collection/Browse.InteractiveSchedule.VerticalCollection+ViewController.swift @@ -18,10 +18,9 @@ private enum Constants { - static let invalidateCellDelay: TimeInterval = 0.1 + static let invalidateCellDebounceTime: TimeInterval = 0.05 static let topCellOffset: Double = 60 } private var focusState: FocusState = .into - private var scrollingDirection: ScrollingDirection? = .none private var animateAfterPosY: Double? @@ -166,17 +165,8 @@ func scrollViewDidScroll(_ scrollView: UIScrollView) { guard let currentFocusedIndexPath, - let scrollingDirection, shouldInvalidate(yPos: scrollView.contentOffset.y) else { return } - let delay: Double = switch scrollingDirection { - case .down: - 0 - case .up: - Constants.invalidateCellDelay - } - - invalidateCellForFocusedIndexPath(currentFocusedIndexPath, after: delay) - self.scrollingDirection = .none + invalidateCellForFocusedIndexPath(currentFocusedIndexPath) self.animateAfterPosY = nil } @@ -193,15 +183,10 @@ let scrollDirectionDifference = targetOffset - scrollView.contentOffset.y - scrollingDirection = scrollDirectionDifference > 0 ? .down : .up animateAfterPosY = scrollView.contentOffset.y + scrollDirectionDifference / 2 } - func invalidateCellForFocusedIndexPath(_ indexPath: IndexPath, after duration: TimeInterval) { + func invalidateCellForFocusedIndexPath(_ indexPath: IndexPath) { debounceInvalidateCellWorkItem?.cancel() - guard duration > .zero else { - collectionView.cellForItem(at: indexPath)?.invalidateIntrinsicContentSize() - return - } - + let workItem = DispatchWorkItem { [weak self] in self?.collectionView.cellForItem(at: indexPath)?.invalidateIntrinsicContentSize() @@ -210,5 +195,5 @@ debounceInvalidateCellWorkItem = workItem DispatchQueue.main.asyncAfter( - deadline: .now() + duration, + deadline: .now() + Constants.invalidateCellDebounceTime, execute: workItem )"},
], tokenize = False, add_generation_prompt = True)

from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 0.8,
    top_p = 0.95,
    max_tokens = 1024,
)
output = model.fast_generate(
    [text],
    sampling_params = sampling_params,
    lora_request = None,
)[0].outputs[0].text

output

Processed prompts: 100%|██████████| 1/1 [00:10<00:00, 10.56s/it, est. speed input: 51.99 toks/s, output: 96.97 toks/s]


"The changes you've made to the code snippet you provided are focused on improving the performance and robustness of the interactive scrolling feature for a vertical collection view. Here's a breakdown of the changes and their implications:\n\n### Original Code Snippet:\nThe original code snippet was dealing with the state management of a vertical collection view's cell invalidation based on the scrolling direction and offset. It was using a delay (`invalidateCellDelay`) to debounce cell invalidation, ensuring that a cell wasn't constantly re-invalidated if the user scrolls very quickly.\n\n### Changes You Made:\n1. **Changed `invalidateCellDelay` to `invalidateCellDebounceTime`**:\n   ```swift\n   static let invalidateCellDebounceTime: TimeInterval = 0.05\n   ```\n   This change is more descriptive, as `invalidateCellDebounceTime` better conveys what this value is used for, which is to debounce the cell invalidation.\n\n2. **Removed the `delay` calculation**:\n   ```swift\n   let dela

And now with the LoRA we just trained with GRPO - we first save the LoRA first!

In [ ]:
model.save_lora("grpo_saved_lora")

Now we load the LoRA and test:

In [41]:
text = tokenizer.apply_chat_template([
    {"role" : "system", "content" : SYSTEM_PROMPT},
    {"role" : "user", "content" : "diff --git a/Project/Scenes/Browse/InteractiveSchedule/Vertical Collection/Browse.InteractiveSchedule.VerticalCollection+ViewController.swift b/Project/Scenes/Browse/InteractiveSchedule/Vertical Collection/Browse.InteractiveSchedule.VerticalCollection+ViewController.swift --- a/Project/Scenes/Browse/InteractiveSchedule/Vertical Collection/Browse.InteractiveSchedule.VerticalCollection+ViewController.swift +++ b/Project/Scenes/Browse/InteractiveSchedule/Vertical Collection/Browse.InteractiveSchedule.VerticalCollection+ViewController.swift @@ -18,10 +18,9 @@ private enum Constants { - static let invalidateCellDelay: TimeInterval = 0.1 + static let invalidateCellDebounceTime: TimeInterval = 0.05 static let topCellOffset: Double = 60 } private var focusState: FocusState = .into - private var scrollingDirection: ScrollingDirection? = .none private var animateAfterPosY: Double? @@ -166,17 +165,8 @@ func scrollViewDidScroll(_ scrollView: UIScrollView) { guard let currentFocusedIndexPath, - let scrollingDirection, shouldInvalidate(yPos: scrollView.contentOffset.y) else { return } - let delay: Double = switch scrollingDirection { - case .down: - 0 - case .up: - Constants.invalidateCellDelay - } - - invalidateCellForFocusedIndexPath(currentFocusedIndexPath, after: delay) - self.scrollingDirection = .none + invalidateCellForFocusedIndexPath(currentFocusedIndexPath) self.animateAfterPosY = nil } @@ -193,15 +183,10 @@ let scrollDirectionDifference = targetOffset - scrollView.contentOffset.y - scrollingDirection = scrollDirectionDifference > 0 ? .down : .up animateAfterPosY = scrollView.contentOffset.y + scrollDirectionDifference / 2 } - func invalidateCellForFocusedIndexPath(_ indexPath: IndexPath, after duration: TimeInterval) { + func invalidateCellForFocusedIndexPath(_ indexPath: IndexPath) { debounceInvalidateCellWorkItem?.cancel() - guard duration > .zero else { - collectionView.cellForItem(at: indexPath)?.invalidateIntrinsicContentSize() - return - } - + let workItem = DispatchWorkItem { [weak self] in self?.collectionView.cellForItem(at: indexPath)?.invalidateIntrinsicContentSize() @@ -210,5 +195,5 @@ debounceInvalidateCellWorkItem = workItem DispatchQueue.main.asyncAfter( - deadline: .now() + duration, + deadline: .now() + Constants.invalidateCellDebounceTime, execute: workItem )"},
], tokenize = False, add_generation_prompt = True)

from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 0.8,
    top_p = 0.95,
    max_tokens = 1024,
)
output = model.fast_generate(
    text,
    sampling_params = sampling_params,
    lora_request = model.load_lora("grpo_saved_lora"),
)[0].outputs[0].text

output

Processed prompts: 100%|██████████| 1/1 [00:01<00:00,  1.19s/it, est. speed input: 656.00 toks/s, output: 62.00 toks/s]


'<reasoning>\nThe commit introduces changes to a view controller class. Notable changes include renaming `invalidateCellDelay` to `invalidateCellDebounceTime`, simplifying the `invalidateCellForFocusedIndexPath` method, and adding debounce functionality.\n</reasoning>\n<answer>\n```git-commit-message\nAdd debounce functionality and simplify method\n```\n</answer>'

Our reasoning model is much better - it's not always correct, since we only trained it for an hour or so - it'll be better if we extend the sequence length and train for longer!

<a name="Save"></a>
### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [ ]:
# Merge to 16bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False: model.save_pretrained_merged("model", tokenizer, save_method = "lora",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "lora", token = "")

### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now! We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list on our [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

[**NEW**] To finetune and auto export to Ollama, try our [Ollama notebook](https://colab.research.google.com/drive/1WZDi7APtQ9VsvOrQSSC5DDtxq159j8iZ?usp=sharing)

In [ ]:
# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)
# Remember to go to https://huggingface.co/settings/tokens for a token!
# And change hf to your username!
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if True: model.push_to_hub_gguf("Tavernari/git-commit-message", tokenizer, quantization_method = "q4_k_m", token = "")

# Save to multiple GGUF options - much faster if you want multiple!
if True:
    model.push_to_hub_gguf(
        "Tavernari/git-commit-message", # Change hf to your username!
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = "",
    )

Now, use the `model-unsloth.gguf` file or `model-unsloth-Q4_K_M.gguf` file in llama.cpp or a UI based system like Jan or Open WebUI. You can install Jan [here](https://github.com/janhq/jan) and Open WebUI [here](https://github.com/open-webui/open-webui)

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/unsloth) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Llama 3.2 Conversational notebook. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(1B_and_3B)-Conversational.ipynb)
2. Saving finetunes to Ollama. [Free notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)
3. Llama 3.2 Vision finetuning - Radiography use case. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(11B)-Vision.ipynb)
6. See notebooks for DPO, ORPO, Continued pretraining, conversational finetuning and more on our [documentation](https://docs.unsloth.ai/get-started/unsloth-notebooks)!

<div class="align-center">
  <a href="https://unsloth.ai"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a>

  Join Discord if you need help + ⭐️ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐️
</div>
